In [1]:
using JSON,Knet,JLD
include(Pkg.dir("Knet","data","imagenet.jl"));
include(Pkg.dir("Knet","examples","vgg","vgg.jl"));

In [2]:
const LAYER_TYPES = ["conv", "relu", "pool", "fc", "prob"]
const numClass=12

12

In [3]:
#batches the data into train test and validation prepares vectors of 224,224,3,n
function batch(model,dict,allkeys)
    imageSize= convert(Array{Int32},(model["meta"]["normalization"]["imageSize"]))
    a,b,c,d=imageSize
    leng=length(allkeys)
    Data=atype(zeros(a,b,c,leng))
    rng = MersenneTwister(1234);
    shuffledKeys=shuffle!(rng, allkeys)
    #Data=zeros(a,b,c,leng)
    trnData,tstData,valData=Any[],Any[],Any[]
    #Train Test and Validation 7:2:1 ratio 
    datasizes=[div(leng*7,10),div(leng*2,10),div(leng,10)]
    xtrn,ytrn=zeros(a,b,c,datasizes[1]),zeros(datasizes[1],1) #train data
    xtst,ytst=zeros(a,b,c,datasizes[2]),zeros(datasizes[2],1)#test data 
    xvl,yvl=zeros(a,b,c,datasizes[3]),zeros(datasizes[3],1)#validation data
    #ygold=zeros(leng,1)
    #Data=convert(atype,zeros(a,b,c,d))
    for (i,key) in enumerate(allkeys)
        d=get(dict,key,nothing)
        x=get(d,"image",nothing)
        y=get(d,"class",nothing)
        #Push to train data
        if i<=datasizes[1]   
            xtrn[:,:,:,i]=x
            ytrn[i,1]=y
        #Push to test data
        elseif i<=datasizes[1]+datasizes[2]  
            xtst[:,:,:,i-datasizes[1]]=x
            ytst[i-datasizes[1],1]=y
        #Push to validation data,
        else
            xvl[:,:,:,i-(datasizes[1]+datasizes[2])]=x
            yvl[i-(datasizes[1]+datasizes[2]),1]=y
        end
    end
    trnData=[xtrn,ytrn]
    tstData=[xtst,ytst]
    valData=[xvl,yvl]
    return trnData,tstData,valData
end 

batch (generic function with 1 method)

In [4]:
function minibatch(data,batchsize,atype)
    batches = Any[]
    xdt,ydt=data
    for k = 1:batchsize:size(xdt,4)
        lo = k
        hi = min(k+batchsize-1, size(xdt,4))        
        samples_x = xdt[:,:,:,lo:hi]
        samples_y = ydt[lo:hi]
        push!(batches,[atype(samples_x),samples_y])
    end
    return batches
end

minibatch (generic function with 1 method)

In [5]:
#https://github.com/denizyuret/Knet.jl/blob/master/examples/vgg/vgg.jl
#Initialize last 3 fc layer weights with xavier and zeroes 
function get_params_with_fc(CNN, atype; last_layer="prob",numberfc=3)
    layers = CNN["layers"]
    weights, operations, derivatives = [], [], []
    countfc=0
    for l in layers
        get_layer_type(x) = startswith(l["name"], x)
        operation = filter(x -> get_layer_type(x), LAYER_TYPES)[1]
        #if operation!="fc"
            push!(operations, operation)
            push!(derivatives, haskey(l, "weights") && length(l["weights"]) != 0)
            if derivatives[end]
                w = copy(l["weights"])
                if operation == "conv"
                    w[2] = reshape(w[2], (1,1,length(w[2]),1))
                elseif operation == "fc"
                    countfc+=1
                    a,b,c,d=size(w[1])
                    e,f=size(w[2])
                    if countfc==numberfc
                        e=numClass
                        d=numClass
                    end
                    w[1] = transpose(mat(xavier(a,b,c,d)))#transpose(mat(w[1]))
                    w[2]=zeros(e,f)
                end
                push!(weights, w)
            end
        last_layer != nothing && get_layer_type(last_layer) && break
    end
    map(w -> map(wi->convert(atype,wi), w), weights), operations, derivatives
end

get_params_with_fc (generic function with 1 method)

In [6]:
function conlayer(weight,x)
    for w in weight
        x=conv4(w[1],x,padding=1).+w[2]
    end
    x=pool(relu.(x))
    return x
end
function fclayer(weight,x)
    for i in 1:length(weight)-1
        x=relu.(weight[i][1]*mat(x).+weight[i][2])
    end
    x=weight[end][1]*mat(x).+weight[end][2]
    return x
end
function predict(weight,x)
    x=conlayer(weight[1:2],x)
    x=conlayer(weight[3:4],x)
    x=conlayer(weight[5:7],x)
    x=conlayer(weight[8:10],x)
    x=conlayer(weight[11:13],x)
    x=fclayer(weight[14:16],x)
    return x
end

predict (generic function with 1 method)

In [7]:
function accuracy(w,data,predfun)
    ntotal=0
    ncorrect=0
    for (x,y) in data
        ntotal+=length(y)
        p=predfun(w,x)
        p=convert(Array,p)
        p=mapslices(indmax,p,1)
        ncorrect += sum( p .== y)  
    end
    return ncorrect/ntotal
end


accuracy (generic function with 1 method)

In [8]:
softmax(w,data,predfun)=mean(softmax(w,x,y,predfun) for (x,y) in data)
function softmax(w,x,label,predfun)
    y1=predfun(w,x)
   # y1=Array{Float32,2}(y1)
    label=Array{Int32}(label)
    loss=0
    for i in 1:size(y1,2)
   #     loss+=-y1[label[i],i]+logp(sum(exp.(y1[:,i])))
        loss+=y1[label[i]]
    end
    return loss
end
softgradloss = gradloss(softmax)

(::gradfun) (generic function with 1 method)

In [18]:
function train(w,dtrn, actfun=sigm; optim=nothing, epochs=10, lr=0.15)
    l=0
    for epoch in 1:epochs
        for (x,y) in dtrn  # dtrn is a list of minibatches
            g,l = softgradloss(w, x, y,predict)
            if(optim==nothing)
                update!(w,g,lr=lr)
            else
                update!(w,g,optim)
            end
        end
        @printf("epoch %d finished, loss=%f\n", epoch, l)
    end
    return l
end

train (generic function with 2 methods)

In [10]:
atype=KnetArray{Float32}
datadict=load("data.jld")["data"];
dataKeys=collect(keys(datadict));
model="imagenet-vgg-verydeep-16" 
model = Main.matconvnet(model)
trn,tst,vl=batch(model,datadict,dataKeys)
batches=minibatch(trn,10,atype)
w,op,der = get_params_with_fc(model, atype)
xtrn,ytrn=batches[1]
pred=predict(w,xtrn)
accuracy(w,batches,predict)
softmax(w,batches,predict)

INFO: Loading imagenet-vgg-verydeep-16.mat...


703.41815f0

In [19]:
lo=train(w,batches, optim=nothing)

epoch 1 finished, loss=NaN
epoch 2 finished, loss=NaN
epoch 3 finished, loss=NaN
epoch 4 finished, loss=NaN
epoch 5 finished, loss=NaN
epoch 6 finished, loss=NaN
epoch 7 finished, loss=NaN
epoch 8 finished, loss=NaN
epoch 9 finished, loss=NaN
epoch 10 finished, loss=NaN


NaN32